In [1]:
import pandas as pd

from pandas import DataFrame as df
import numpy as np
import csv

In [2]:
otu = pd.read_csv('data/silva/freshwater/emp_silva_freshwater.txt', sep = '\t', index_col = 0)
print(otu.shape)

(77862, 5817)


In [3]:
import re
otu_ids = []
for i in otu.index.values:
    otu_ids.append(re.sub("\.(.)*", "", i))
print("OTUs relabeled")


OTUs relabeled


In [4]:
rename = {}
for i in range(otu.shape[0]):
    rename[otu.index.values[i]]= otu_ids[i]
otu = otu.rename(rename)
otu.head()
#OTUs renamed

,1039.P.Dois.Rios.SA,1039.L.Jacarepia.HB,1041.S008.2m.off.bottom,1041.M041.2m.off.bottom,1041.S008.10,1041.S008.150,1041.S008.200,1041.S019.100,1041.S019.20,1041.S019.5,...,945.P12.H4.lane5.NoIndex.L005,945.P2.G1.lane2.NoIndex.L002,945.P5.G4.lane2.NoIndex.L002,945.P14.E12.lane3.NoIndex.L003,945.P12.G6.lane5.NoIndex.L005,945.P6.H3.lane2.NoIndex.L002,945.P7.G4.lane2.NoIndex.L002,945.P6.B4.lane2.NoIndex.L002,945.P7.H10.lane2.NoIndex.L002,945.P7.G10.lane2.NoIndex.L002
#OTU ID,,,,,,,,,,,,,,,,,,,,,
FJ478961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JQ919464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AJXC01007386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JQ825201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KJ145789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
#Only useful for reducing data size when using already subsetted data. If using whole dataset, all taxa are present at least once
#total_taxa_abund = otu.sum(axis = 1) #Should be ntaxa (5000) of them
#otu_prune = otu.loc[total_taxa_abund > 0, :]
#print("After deleting taxa that never appear, we have " + str(otu_prune.shape[0]) + " taxa in " + str(otu_prune.shape[1]) + " samples")

Only keep taxa present in greater than 58.17 samples
We will keep 18567 taxa


In [10]:
#I want to exclude the last study from embedding formation, to be used as a test set
#Read mapping data
mapping = pd.read_csv("data/emp_qiime_mapping_release1.tsv.csv", sep = ",", index_col=0, encoding='latin1')
print("Mapping has shape: " + str(mapping.shape))

map_filt = mapping.loc[mapping['empo_3'] == "Water (non-saline)"]
print("After selecting for biome, mapping has shape " + str(map_filt.shape))

bools = [i in otu.columns.values for i in map_filt.index.values]
map_filt = map_filt.loc[bools]
print("After selecting just the samples present in the otu table: " + str(map_filt.shape))

otu_sort = otu.reindex(sorted(otu.columns.values), axis = 1)
map_filt_sort = map_filt.reindex(sorted(map_filt.index.values), axis = 0)

nsamples = map_filt.shape[0]
bools_correct = [map_filt_sort.index.values[i] == otu_sort.columns.values[i] for i in range(nsamples)]
print("After rearranging, we have " + str(np.sum(bools_correct)) + " matching samples")
if (np.sum(bools_correct) == map_filt_sort.shape[0]) and (np.sum(bools_correct) == otu_sort.shape[1]):
    print("Safe to continue")
else:
    print("STOP! Something is wrong.")
    

Mapping has shape: (27751, 75)
After selecting for biome, mapping has shape (5940, 75)
After selecting just the samples present in the otu table: (5817, 75)
After rearranging, we have 5817 matching samples
Safe to continue


In [14]:
mapping = map_filt_sort
otu = otu_sort.loc[:, mapping.study_id != 1883]

thresh = 0.02 * otu.shape[1] # at least 1 % of samples (10 in this case)
print("Only keep taxa present in greater than " + str(thresh) + " samples")
binary = otu > 0
keep = binary.sum(axis = 1) >= thresh# Should be ntaxa
binary = binary.loc[keep, :]

print("We will keep " + str(keep.sum()) +" taxa")
print(binary.shape)

Only keep taxa present in greater than 64.38 samples
We will keep 11132 taxa
(11132, 3219)


In [21]:
np.sum(binary)
#otu_filter.to_csv("data/silva/freshwater/otu_filtered_freshwater_5perc.csv", sep = "\t", index = True)

1039.L.Jacarepia.HB               963
1039.P.Dois.Rios.SA              1133
1041.M028.10                      985
1041.M028.100                     936
1041.M028.10m.off.bottom          855
1041.M028.20                      812
1041.M028.2m.off.bottom           996
1041.M028.5                       978
1041.M028.50                      945
1041.M041.10                      632
1041.M041.100                     552
1041.M041.10m.off.bottom          704
1041.M041.150                     648
1041.M041.20                      710
1041.M041.200                     697
1041.M041.2m.off.bottom           710
1041.M041.5                       722
1041.M041.50                      555
1041.S001.10                      431
1041.S001.10m.off.bottom          842
1041.S001.20                      360
1041.S001.2m.off.bottom          1046
1041.S001.5                       753
1041.S001.50                      924
1041.S008.10                      757
1041.S008.100                     742
1041.S008.10

In [22]:
file_text = []
for i in range(binary.shape[1]):
    taxa_share = binary.index[binary.iloc[:,i] == True]
    file_text.append(taxa_share)

Index(['EF019723', 'HM274550', 'EF020288', 'JN541129', 'EU861855', 'CU925012',
       'GQ202666', 'DQ149125', 'HM270195', 'FR846982',
       ...
       'EF492972', 'FQ659178', 'AB486865', 'EU777724', 'HE648209', 'KC432217',
       'HQ730668', 'EU773316', 'KM205539', 'DQ129399'],
      dtype='object', name='#OTU ID', length=963)
Index(['EF019723', 'FJ592820', 'JQ426168', 'EU802980', 'JQ978525', 'HQ118319',
       'HQ397514', 'HM187082', 'FQ659743', 'DQ149125',
       ...
       'AM176779', 'KC290430', 'AB486437', 'HQ178842', 'AJ401107', 'FJ208805',
       'FJ638599', 'KC424696', 'GU180177', 'JN397961'],
      dtype='object', name='#OTU ID', length=1133)
Index(['EU800310', 'EF157273', 'EF590039', 'JN541145', 'DQ501327', 'HM069087',
       'EU644179', 'HM822554', 'AY218755', 'AY188315',
       ...
       'AF172715', 'KC002982', 'KC253289', 'GQ340109', 'FJ745191', 'AF316775',
       'HE574381', 'GQ231372', 'JQ941846', 'EF590007'],
      dtype='object', name='#OTU ID', length=985)
Index(['E

Index(['HM459665', 'HQ118319', 'FN400860', 'AY345582', 'EF532600', 'GU731324',
       'HQ532204', 'AJ867655', 'AY948052', 'KC961291',
       ...
       'JN656764', 'AY792309', 'KJ186926', 'KF061594', 'JN038810', 'JQ919567',
       'AB930542', 'JF917207', 'AF524854', 'EF520445'],
      dtype='object', name='#OTU ID', length=498)
Index(['EF532600', 'HQ532204', 'EU037328', 'GQ074238', 'HF933203', 'KJ123792',
       'GQ076390', 'FR823006', 'EU104100', 'LN565405',
       ...
       'AY792260', 'JF327660', 'KF827317', 'AJ575702', 'AY792224', 'EU803296',
       'JN656764', 'AY792309', 'GU135465', 'AF524854'],
      dtype='object', name='#OTU ID', length=333)
Index(['KC157042', 'LN571267', 'EF532600', 'HQ532204', 'GU319965', 'EF516172',
       'CU923685', 'AY792311', 'FR823006', 'LN565405',
       ...
       'KC253294', 'JX222300', 'JN626567', 'GQ390232', 'JN867670', 'GU127227',
       'JQ919567', 'JF917207', 'JN656908', 'JN038990'],
      dtype='object', name='#OTU ID', length=433)
Index(['KC

Index(['JF224558', 'HM069087', 'KJ600912', 'FN400860', 'FR846982', 'AY188315',
       'LN571267', 'JX223421', 'HQ532204', 'EF516172',
       ...
       'KC502892', 'FQ660014', 'AB077222', 'FQ659595', 'GU135463', 'DQ676456',
       'Y08103', 'HM481321', 'GU389881', 'HM487998'],
      dtype='object', name='#OTU ID', length=1200)
Index(['JF224558', 'HM069087', 'FJ437989', 'FR846982', 'AY188315', 'LN571267',
       'HQ532204', 'EF516172', 'CU923685', 'AB821062',
       ...
       'JQ919723', 'FJ628275', 'JN656908', 'AB077222', 'FQ659595', 'GU135463',
       'DQ676456', 'Y08103', 'HM481321', 'GU389881'],
      dtype='object', name='#OTU ID', length=828)
Index(['JF208044', 'JF224558', 'HM069087', 'FJ437989', 'LN571267', 'JX223421',
       'JF168275', 'FR691490', 'JN650265', 'AY792311',
       ...
       'JQ919723', 'FJ628275', 'JX225064', 'JN656908', 'AB077222', 'FQ659595',
       'GU135463', 'HM481321', 'GU389881', 'HM487998'],
      dtype='object', name='#OTU ID', length=635)
Index(['JF224

Index(['KC157042', 'FR846982', 'LN571267', 'EF532600', 'FJ628183', 'HQ532204',
       'KC620879', 'EF520480', 'HM773508', 'FR691490',
       ...
       'JQ919723', 'FJ628275', 'EF520445', 'AB077222', 'KM410667', 'GU135463',
       'DQ676456', 'HM481321', 'GU389881', 'HM487998'],
      dtype='object', name='#OTU ID', length=774)
Index(['KC157042', 'HM822554', 'FJ437989', 'FR846982', 'AY188315', 'LN571267',
       'JN084135', 'JX223421', 'HQ532204', 'KC620879',
       ...
       'EF520445', 'JX225064', 'JN656908', 'KC502892', 'FQ660014', 'AB077222',
       'DQ676456', 'HM481321', 'GU389881', 'HM487998'],
      dtype='object', name='#OTU ID', length=1278)
Index(['KC157042', 'KJ615139', 'FR846982', 'AY188315', 'LN571267', 'EF532600',
       'GU731324', 'HQ532204', 'EF516172', 'JF139763',
       ...
       'AB930542', 'AF351220', 'JQ919723', 'FJ628275', 'JX225064', 'JN656908',
       'AB077222', 'HM481321', 'GU389881', 'KF911225'],
      dtype='object', name='#OTU ID', length=921)
Index(['K

      dtype='object', name='#OTU ID', length=322)
Index(['EU861954', 'LN571267', 'AM936813', 'KC961291', 'AB821062', 'JQ855569',
       'JF176936', 'AY792311', 'GQ076390', 'AY792276',
       ...
       'DQ145977', 'JQ279017', 'GU127220', 'AY792231', 'EF520350', 'EF520568',
       'AY792260', 'AY792224', 'JN626567', 'JF917207'],
      dtype='object', name='#OTU ID', length=222)
Index(['KC157042', 'HM069087', 'FR846982', 'LN571267', 'HQ532204', 'EF516172',
       'HM773508', 'HM111704', 'KC961291', 'JF176936',
       ...
       'KC620906', 'JN626567', 'GQ390232', 'JN697897', 'JN867670', 'AB722234',
       'FJ628275', 'EF520445', 'Y08103', 'GU731321'],
      dtype='object', name='#OTU ID', length=460)
Index(['FR846982', 'LN571267', 'EF520480', 'JF176936', 'FR691490', 'KJ123792',
       'AY792311', 'FR823006', 'AY792276', 'AY792258',
       ...
       'JX223659', 'HM481333', 'FJ502264', 'EU803296', 'EU700148', 'AY792309',
       'FJ716275', 'JN626567', 'FJ628275', 'GU731321'],
      dtype=

...

Index(['JQ919464', 'KC920982', 'KF366398', 'HM822554', 'AY345582', 'JN084135',
       'EU465875', 'KC000085', 'FJ628183', 'FJ966157',
       ...
       'AF443582', 'EU474723', 'EU777849', 'EU777835', 'EF445197', 'EF445153',
       'EU777879', 'EU462774', 'DQ129594', 'EU458664'],
      dtype='object', name='#OTU ID', length=317)
Index(['HQ256538', 'KC920982', 'EU375216', 'JX133400', 'KF366398', 'EU369167',
       'HM822554', 'JN084135', 'EU465875', 'KC000085',
       ...
       'DQ172992', 'JX079371', 'JQ855581', 'GQ915089', 'EU772078', 'EU772214',
       'HM277807', 'KC110933', 'DQ129594', 'EU772203'],
      dtype='object', name='#OTU ID', length=403)
Index(['HM129985', 'KF836169', 'HM069087', 'HM822554', 'AY188315', 'GQ076271',
       'DQ125624', 'KC442629', 'AJ867655', 'CU923685',
       ...
       'KF827369', 'HW339955', 'JN941764', 'JX006087', 'JF766568', 'FQ659710',
       'AB599879', 'FN668114', 'AJ007864', 'GU293203'],
      dtype='object', name='#OTU ID', length=629)
Index(['KC

Index(['EU800310', 'HM129985', 'EF157273', 'JN541145', 'KF836169', 'DQ501327',
       'HM069087', 'AY218755', 'AY188315', 'LN571267',
       ...
       'X81182', 'GQ340324', 'AB930723', 'FQ660142', 'EU803296', 'FQ659010',
       'AB599879', 'FN668114', 'FR667347', 'DQ501340'],
      dtype='object', name='#OTU ID', length=781)
Index(['KC157042', 'JF703415', 'EU800310', 'HM129985', 'JN541145', 'KF836169',
       'HM069087', 'HM822554', 'AY218755', 'LN571267',
       ...
       'GU127213', 'KJ600854', 'JF429367', 'FR798962', 'GQ340324', 'AB930723',
       'FQ658607', 'FQ660142', 'AB599879', 'KM410467'],
      dtype='object', name='#OTU ID', length=719)
Index(['KC157042', 'EU800310', 'JF148534', 'HM129985', 'EF157273', 'JN541145',
       'KF836169', 'DQ501327', 'HM069087', 'AB051695',
       ...
       'JX221764', 'FJ820382', 'DQ521525', 'FJ755705', 'GQ340324', 'AB930723',
       'FQ660142', 'FQ659010', 'FN668114', 'FJ208810'],
      dtype='object', name='#OTU ID', length=990)
Index(['EU80

Index(['EU800310', 'HM129985', 'KF836169', 'DQ501327', 'HM069087', 'AY218755',
       'AY188315', 'LN571267', 'KF712541', 'HW522589',
       ...
       'KC358192', 'KC253327', 'GQ340324', 'AB930723', 'EU803296', 'AB991298',
       'EF580973', 'GQ339153', 'JQ428278', 'GU293203'],
      dtype='object', name='#OTU ID', length=914)
Index(['EU800310', 'JF148534', 'HM129985', 'KC442489', 'HM069087', 'AB051695',
       'HQ190569', 'EU800274', 'KC442629', 'KC618496',
       ...
       'DQ676309', 'JX221764', 'FJ820382', 'AB935929', 'JX006087', 'FN824903',
       'HM481333', 'FQ659891', 'FR667347', 'Y17601'],
      dtype='object', name='#OTU ID', length=677)
Index(['EU800310', 'HM129985', 'KF836169', 'EF019478', 'DQ501327', 'HM069087',
       'AY218755', 'AY188315', 'LN571267', 'EU800274',
       ...
       'FQ660142', 'AY792309', 'AB599879', 'KF463375', 'AB661544', 'JN626567',
       'AB754152', 'EU474723', 'GU293203', 'DQ501340'],
      dtype='object', name='#OTU ID', length=848)
Index(['EU80

Index(['DQ664245', 'HM822554', 'FN400860', 'LN571267', 'JN084135', 'FQ658779',
       'JX102501', 'AM936813', 'HW522589', 'FJ889302',
       ...
       'HM154512', 'FJ466346', 'EU801522', 'JN867670', 'JX114401', 'JQ919567',
       'GQ339168', 'JF958134', 'EF520445', 'JF917220'],
      dtype='object', name='#OTU ID', length=647)
Index(['KC157042', 'JF703415', 'HM129985', 'JF208044', 'KF366398', 'JF175099',
       'HM069087', 'HM822554', 'AB051695', 'AY218755',
       ...
       'JX222300', 'JX114401', 'JF917207', 'GQ339168', 'AF524854', 'JF958134',
       'EF520445', 'HM481397', 'EU133674', 'JF917220'],
      dtype='object', name='#OTU ID', length=1111)
Index(['KC157042', 'DQ664245', 'KC920982', 'EF019124', 'AB051695', 'EU835406',
       'AY188315', 'LN571267', 'EF532600', 'JN084135',
       ...
       'EU700148', 'FQ660486', 'AY792309', 'JN038810', 'JN626567', 'JF958134',
       'HE604035', 'JN656908', 'JX225787', 'GQ448452'],
      dtype='object', name='#OTU ID', length=955)
Index(['K

Index(['KC920982', 'EU375216', 'JN541129', 'KF366398', 'JF175099', 'FJ687978',
       'JF968481', 'FJ375397', 'HM822554', 'AB051695',
       ...
       'JN167608', 'JN695619', 'JF169117', 'EU431820', 'KF084899', 'DQ172992',
       'JF216730', 'EU133674', 'HM277807', 'KC110933'],
      dtype='object', name='#OTU ID', length=653)
Index(['KC157042', 'DQ664245', 'HM129985', 'KF037771', 'FN400860', 'AY188315',
       'LN571267', 'JX102501', 'AM936813', 'HQ843705',
       ...
       'KM410467', 'FQ659891', 'JN626567', 'JF176782', 'GQ339168', 'EF520445',
       'JN656908', 'GU731321', 'KC110933', 'JF917220'],
      dtype='object', name='#OTU ID', length=918)
Index(['KF366398', 'EU369167', 'EU465962', 'JN084135', 'EU465875', 'KC000085',
       'JX223421', 'AB680327', 'KJ600989', 'JF920346',
       ...
       'EU773141', 'HQ413057', 'EU474931', 'GQ448437', 'EU779247', 'GQ448452',
       'EU466217', 'EU470458', 'FJ848438', 'EU472427'],
      dtype='object', name='#OTU ID', length=249)
Index(['EU

In [17]:
lengths = []
for taxa_list in file_text:
    lengths.append(len(taxa_list))
max_val = max(lengths)
max_ind = np.argmax(lengths)
print("Max number of taxa in any given sample is: " + str(max_val))
print("Found at " + str(max_ind))


Max number of taxa in any given sample is: 4425
Found at 146


In [23]:
#Write Glove input file
with open('data/silva/freshwater/glove_input_freshwater_2perc.txt', mode = 'w', newline='') as file:
    writer = csv.writer(file, delimiter = "\t", quoting = csv.QUOTE_NONE, escapechar = '')
    for taxa_list in file_text:
        print(taxa_list)
        writer.writerow(taxa_list)
file.close()


Index(['EF019723', 'HM274550', 'EF020288', 'JN541129', 'EU861855', 'CU925012',
       'GQ202666', 'DQ149125', 'HM270195', 'FR846982',
       ...
       'EF492972', 'FQ659178', 'AB486865', 'EU777724', 'HE648209', 'KC432217',
       'HQ730668', 'EU773316', 'KM205539', 'DQ129399'],
      dtype='object', name='#OTU ID', length=963)
Index(['EF019723', 'FJ592820', 'JQ426168', 'EU802980', 'JQ978525', 'HQ118319',
       'HQ397514', 'HM187082', 'FQ659743', 'DQ149125',
       ...
       'AM176779', 'KC290430', 'AB486437', 'HQ178842', 'AJ401107', 'FJ208805',
       'FJ638599', 'KC424696', 'GU180177', 'JN397961'],
      dtype='object', name='#OTU ID', length=1133)
Index(['EU800310', 'EF157273', 'EF590039', 'JN541145', 'DQ501327', 'HM069087',
       'EU644179', 'HM822554', 'AY218755', 'AY188315',
       ...
       'AF172715', 'KC002982', 'KC253289', 'GQ340109', 'FJ745191', 'AF316775',
       'HE574381', 'GQ231372', 'JQ941846', 'EF590007'],
      dtype='object', name='#OTU ID', length=985)
Index(['E

Index(['HM459665', 'HQ843705', 'HQ532204', 'KC961291', 'AY792311', 'JQ311876',
       'KJ161326', 'FR823006', 'EU104100', 'KF101300',
       ...
       'AY792260', 'KF827317', 'AJ575702', 'AY792224', 'EU803296', 'AY792309',
       'FJ482228', 'GU127227', 'JQ919567', 'JF917207'],
      dtype='object', name='#OTU ID', length=291)
Index(['KC157042', 'DQ664245', 'FN400860', 'FR846982', 'LN571267', 'JX223421',
       'JX102501', 'HQ532204', 'GU319965', 'AJ867655',
       ...
       'JN697897', 'GU127227', 'JQ919567', 'JF917207', 'DQ202246', 'EF520445',
       'JN656908', 'Y08103', 'JQ861811', 'GU731321'],
      dtype='object', name='#OTU ID', length=623)
Index(['KC157042', 'DQ664245', 'CU925012', 'LN571267', 'JX102501', 'HQ532204',
       'AUMO01000173', 'HM773508', 'CU923685', 'AY948052',
       ...
       'EU801522', 'JN626567', 'JN697897', 'GU127227', 'JF917207', 'FJ628275',
       'AY626060', 'EF648135', 'JN656908', 'GU731321'],
      dtype='object', name='#OTU ID', length=416)
Index(['

Index(['KC157042', 'JF224558', 'AY188315', 'LN571267', 'EF532600', 'KC442629',
       'HQ532204', 'EF516172', 'KC189665', 'KM525665',
       ...
       'HQ828010', 'AJ575702', 'EU803296', 'JN656764', 'DQ444439', 'AY792309',
       'JN626567', 'EF520445', 'JN656908', 'KF071039'],
      dtype='object', name='#OTU ID', length=580)
Index(['KC157042', 'JF224558', 'KC442629', 'HQ532204', 'EF516172', 'FJ889302',
       'KM525665', 'CU923685', 'FM886874', 'KF037812',
       ...
       'EF520565', 'AY792260', 'HQ828010', 'AJ575702', 'AY792224', 'EU803296',
       'DQ444439', 'AY792309', 'JN626567', 'JN656908'],
      dtype='object', name='#OTU ID', length=439)
Index(['JF224558', 'LN571267', 'AM936813', 'HQ532204', 'KC189665', 'AJ867655',
       'ADOS01000480', 'CU923685', 'AB821062', 'AY162067',
       ...
       'EF520565', 'AY792260', 'EU803296', 'JN656764', 'EU700148', 'AY792309',
       'AB364724', 'HM487983', 'FJ628275', 'JN656908'],
      dtype='object', name='#OTU ID', length=420)
Index(

      dtype='object', name='#OTU ID', length=266)
Index(['KC157042', 'GU179688', 'AY218672', 'LN571267', 'EU134601', 'EF532600',
       'HQ532204', 'KM525665', 'ADOS01000480', 'HM773508',
       ...
       'AY792224', 'JN656764', 'EU700148', 'AY792309', 'JN626567', 'JQ919567',
       'JF958134', 'FJ628275', 'EF520445', 'EF648135'],
      dtype='object', name='#OTU ID', length=470)
Index(['LN571267', 'EF532600', 'HQ532204', 'EF516172', 'ADOS01000480',
       'GAON01156412', 'CU923685', 'HM111704', 'KC961291', 'AB821062',
       ...
       'AY792231', 'EU431820', 'EF520568', 'AY792260', 'AJ575702', 'AY792224',
       'AY792309', 'JN626567', 'EF520445', 'JN656908'],
      dtype='object', name='#OTU ID', length=304)
Index(['LN571267', 'EF532600', 'HQ532204', 'EF516172', 'HM773508', 'HM111704',
       'AY948052', 'KC961291', 'FJ375401', 'KC437172',
       ...
       'KF463375', 'EU528249', 'AB722243', 'JN626567', 'GU127227', 'JQ919567',
       'JF917207', 'FJ628275', 'EF520445', 'JN656908']

      dtype='object', name='#OTU ID', length=552)
Index(['EU861954', 'FJ625344', 'AB051695', 'FN400860', 'FR846982', 'AY188315',
       'AM936813', 'HQ843705', 'HQ532204', 'KC620879',
       ...
       'GQ339168', 'FJ628275', 'EF520445', 'JX225064', 'GU135463', 'Y08103',
       'JQ861811', 'JN038990', 'GU389881', 'GU731321'],
      dtype='object', name='#OTU ID', length=815)
Index(['HQ532204', 'KC618496', 'JF176936', 'AY792311', 'KC554479', 'JQ428090',
       'HQ691997', 'FR823006', 'AJ318192', 'AY792276',
       ...
       'EU700148', 'FJ482178', 'DQ787723', 'AB364724', 'GQ339168', 'EF520445',
       'JX225064', 'KM410667', 'JN038990', 'KF911225'],
      dtype='object', name='#OTU ID', length=224)
Index(['HQ532204', 'JF176936', 'AY792311', 'KC554479', 'HQ691997', 'FR823006',
       'AY792276', 'KF071227', 'AY792258', 'HM773513',
       ...
       'JX223659', 'AY792260', 'KF827317', 'AJ575702', 'EU700148', 'FJ482178',
       'DQ787723', 'AB364724', 'JX225064', 'JN038990'],
      dtype=

...

Index(['KC157042', 'JF703415', 'EU800310', 'JF148534', 'EF020288', 'JF208044',
       'EF019415', 'JQ978525', 'EF590039', 'JX223117',
       ...
       'HM638233', 'AB991298', 'FR667462', 'JF833636', 'HQ697403', 'JQ855581',
       'FR667347', 'JF216730', 'EF520445', 'AB990041'],
      dtype='object', name='#OTU ID', length=2078)
Index(['KJ615139', 'JF703415', 'EU800310', 'JF148534', 'AF506059', 'EU665101',
       'KC920982', 'HM269827', 'EF020288', 'JQ978525',
       ...
       'EU340220', 'FJ593907', 'JF204128', 'FQ660142', 'FR667462', 'HQ697403',
       'AB722234', 'EF520445', 'KF650386', 'AB990041'],
      dtype='object', name='#OTU ID', length=1880)
Index(['KF853113', 'JF703415', 'JF148534', 'EF020288', 'EF019415', 'FJ849470',
       'JQ978525', 'EF590039', 'JF224558', 'JN541145',
       ...
       'AY792260', 'FQ660142', 'AB991298', 'FR667462', 'JF833636', 'HQ697403',
       'FR667347', 'JQ919567', 'EF520445', 'GU731321'],
      dtype='object', name='#OTU ID', length=2042)
Index([

      dtype='object', name='#OTU ID', length=293)
Index(['HQ256538', 'AF506059', 'EU375216', 'KF366398', 'HM822554', 'FN400860',
       'AY345582', 'EF532600', 'JN084135', 'EU465875',
       ...
       'EU458642', 'EU777879', 'EU462644', 'EU462774', 'DQ129594', 'JF211036',
       'EU458600', 'EU462612', 'EU777641', 'EU458664'],
      dtype='object', name='#OTU ID', length=529)
Index(['AY345582', 'EU465875', 'KC000085', 'DQ125624', 'HW522589', 'AJ867655',
       'KM525665', 'JF920346', 'EU179735', 'GU569115',
       ...
       'JX095277', 'EU470458', 'EU461014', 'FJ848438', 'EU461345', 'EF445187',
       'EU468883', 'AY212765', 'JX096274', 'EU472427'],
      dtype='object', name='#OTU ID', length=242)
Index(['KC920982', 'HM822554', 'FN400860', 'AY345582', 'JN084135', 'EU465875',
       'FJ628183', 'AJ867655', 'KM525665', 'JN672347',
       ...
       'EU462644', 'EU462774', 'EU458600', 'EU462612', 'EU458649', 'EU777641',
       'EU458663', 'EU472427', 'EU458664', 'EU458609'],
      dtyp

Index(['KC157042', 'KJ615139', 'EU800310', 'HM129985', 'KC920982', 'JN541145',
       'KF836169', 'EF019478', 'DQ501327', 'HM069087',
       ...
       'KM410344', 'FN824903', 'FQ659710', 'AB630747', 'FQ660142', 'AB364724',
       'AJ007864', 'FJ208810', 'DQ501340', 'JF917220'],
      dtype='object', name='#OTU ID', length=1145)
Index(['EU800310', 'HM129985', 'EF157273', 'JN541145', 'KF836169', 'DQ501327',
       'HM069087', 'AY218755', 'AY188315', 'LN571267',
       ...
       'X81182', 'GQ340324', 'AB930723', 'FQ660142', 'EU803296', 'FQ659010',
       'AB599879', 'FN668114', 'FR667347', 'DQ501340'],
      dtype='object', name='#OTU ID', length=781)
Index(['KC157042', 'JF703415', 'EU800310', 'HM129985', 'JN541145', 'KF836169',
       'HM069087', 'HM822554', 'AY218755', 'LN571267',
       ...
       'GU127213', 'KJ600854', 'JF429367', 'FR798962', 'GQ340324', 'AB930723',
       'FQ658607', 'FQ660142', 'AB599879', 'KM410467'],
      dtype='object', name='#OTU ID', length=719)
Index(['KC1

Index(['AY792258', 'AJ575553', 'AY792309'], dtype='object', name='#OTU ID')
Index(['EF019723', 'EF019415', 'CU925012', 'JF224558', 'HM069087', 'HM822554',
       'AB051695', 'FJ437989', 'AY188315', 'LN571267',
       ...
       'JX225771', 'JN038810', 'JN626567', 'AB930542', 'JF958134', 'AB722234',
       'EF520445', 'JN656908', 'DQ676456', 'GU731321'],
      dtype='object', name='#OTU ID', length=882)
Index(['KC157042', 'FJ382617', 'HM129985', 'KC920982', 'HM822554', 'FN400860',
       'AY188315', 'LN571267', 'JX223421', 'EF516172',
       ...
       'EU528249', 'HM154512', 'FQ659891', 'DQ354723', 'GQ339168', 'AJ290825',
       'AF524854', 'JQ861811', 'JN038990', 'JF917220'],
      dtype='object', name='#OTU ID', length=791)
Index(['KC920982', 'HM069087', 'FN400860', 'FR846982', 'AY188315', 'LN571267',
       'JX102501', 'AM936813', 'HQ532204', 'AJ867655',
       ...
       'GU135465', 'FQ659891', 'JX114401', 'AF523968', 'AM162477', 'JF176782',
       'GQ339168', 'JF958134', 'EF445153

      dtype='object', name='#OTU ID', length=787)
Index(['EF019723', 'KC157042', 'KJ615139', 'KC920982', 'EU375216', 'JN541129',
       'KF366398', 'EU369167', 'FJ375397', 'HM822554',
       ...
       'GQ449038', 'AJ303009', 'KC213491', 'AJ416411', 'EU779229', 'EU800293',
       'JN695619', 'JF169117', 'DQ172992', 'JQ855581'],
      dtype='object', name='#OTU ID', length=639)
Index(['KC920982', 'EU375216', 'JN541129', 'KF366398', 'HM822554', 'EU835406',
       'AY188315', 'EU465875', 'JN990081', 'JX223421',
       ...
       'KM108670', 'EU779196', 'GQ451299', 'AJ303009', 'CU927050', 'JN695619',
       'JF766568', 'JQ855581', 'JF176782', 'EU133674'],
      dtype='object', name='#OTU ID', length=554)
Index(['JN023493', 'KC157042', 'FJ592820', 'KC920982', 'EU375216', 'JN541129',
       'EU369167', 'FJ375397', 'AB051695', 'FN400860',
       ...
       'JF216730', 'JF176782', 'GQ915089', 'JX225787', 'EU133674', 'EU474723',
       'GQ448437', 'JX095582', 'EU470458', 'KC110933'],
      dtyp